#### 1.假设我们输入的矩阵为[5, 5, 3]

In [3]:
import tensorflow as tf
import numpy as np

# 构建[5, 5, 3]的矩阵
M = np.random.randint(-10, 10, size=[5, 5, 3])

# 打印矩阵的数据
print("Matrix shape is: ", M.shape)
print("Matrix data is: ", M.data)

Matrix shape is:  (5, 5, 3)
Matrix data is:  <memory at 0x00000221F9BACD68>


#### 2.定义卷积过滤器，为6@[5, 5, 3]

In [5]:
# 通过tf.get_variable()函数创建过滤器
# 这里申明的参数为4维变量，
# 前两个参数表示过滤尺寸
# 第三个参数表示输入层的深度
# 第四个参数表示过滤器的个数
filter_weight = tf.get_variable("weights",
                               shape=[5, 5, 3, 6],
                               initializer=tf.truncated_normal_initializer(stddev=0.1))
# 设置偏置项
biases = tf.get_variable("biases", shape=[6], 
                         initializer=tf.constant_initializer(0.1))

#### 3.调整输入的格式符合TensorFlow的要求

In [6]:
M = np.asarray(M, dtype='float32') # 调整类型
M = M.reshape(1, 5, 5, 3)

#### 4.计算矩阵通过卷积层过滤和池化层过滤器计算后的结果

In [8]:
# 构建输入节点
# [1, None, None, None]:
# 第一个维度：表示batch的大小
# 后面三个维度：表示图片尺寸的大小
x = tf.placeholder(tf.float32, [1, None, None, None])

# 构建卷积层
# tf.nn.conv2d提供了一个非常方便的函数来实现卷积层前向传播算法。
# 第一个参数：为当前层的节点矩阵。注意这个节点矩阵是一个四维矩阵，
# 后面三维对应一个节点矩阵，第一维对应一个输入batch
# 比如：input[0, :, :, :]表示第一张图片；
# input[1, :, :, :]表示第二张图片；后续以此类推
# 第二个参数提供了卷积层的权重，
# 第三个参数为不同维度上的步长,虽然第三个参数提供的是一个长度为4的数组，
# 但是第一维和最后一维的数字一定是1.这是因为卷积的步长只对矩阵的长宽有效
# 最后一个参数是填充(padding)的方法，TensorFlow中提供SAME或者VALID两种选择。
# 其中SAME表示添加全0填充，VALID表示不添加
conv = tf.nn.conv2d(x, filter_weight, strides=[1, 2, 2, 1], padding='SAME')

# tf.nn.bias_add提供了一个方便的函数给每一个节点加上偏置项。
# 注意这里不能直接使用加法
# 因为矩阵上不同位置的节点上都需要加上同样的偏置项
layer = tf.nn.bias_add(conv, biases)

# 构建池化层
# tf.nn.max_pool实现了最大池化层的前向传播。
# ksize：提供了过滤器的尺寸，四维矩阵，第一个参数与最后一个参数必须为1
# strides：提供了步长信息，四维矩阵，第一个参数和最后一个参数必须为1，
# 常见池化层过滤器尺寸为[1,2,2,1]或者[1,3,3,1]
# padding：提供了是否使用全零填充
# tf.nn.avg_pool实现了平均池化。参数与tf.nn.max_pool()函数一样
pool = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 建立会话
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(config=config) as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    # 计算卷积层
    convoluted_M = sess.run(layer, feed_dict={x:M})
    
    # 计算池化层
    pooled_M = sess.run(pool, feed_dict={x:M})
    
    # 打印结果
    print("convoluted_M.shape: \n", convoluted_M.shape)
    print("convoluted_M: \n", convoluted_M)
    print("pooled_M: \n", pooled_M.shape)
    print("pooled_M: \n", pooled_M)

convoluted_M.shape: 
 (1, 3, 3, 6)
convoluted_M: 
 [[[[-0.10547302  1.73650408  1.06343293  3.40875983 -1.66920519  4.06999254]
   [ 4.56235838  5.49796677  0.80188239  4.80416298 -0.77915525 -1.69816339]
   [ 3.52190471  2.11461258  2.86656785  3.29678679 -1.23762023 -1.59586537]]

  [[ 1.014557    2.0864861   5.71319389  4.29242659  3.35116506 -0.85415018]
   [-4.20453215 -2.21270347 -4.79761028 -0.86295831  1.10380471 -2.40811491]
   [ 3.89522338  0.53519845 -2.33123493 -2.37243772  5.07914209 -2.16623425]]

  [[ 1.36512244 -0.97684944 -1.1289897  -0.38216463 -1.31056905  0.32951298]
   [ 5.47473383 -1.08033419  6.41506433 -0.0331571   2.93214369  3.89121938]
   [ 2.69612193  2.91534138 -7.02439499 -0.0717214   0.88957447 -0.04524925]]]]
pooled_M: 
 (1, 3, 3, 3)
pooled_M: 
 [[[[  8.   6.   5.]
   [  3.   4.   3.]
   [ -1. -10.   9.]]

  [[  1.   9.   7.]
   [  9.   1.   5.]
   [ -1.  -2.   8.]]

  [[ -4.   2.  -5.]
   [  8.  -1.  -2.]
   [ -1.   2.   8.]]]]
